In [1]:
using JuMP, Gurobi

#### 1. Building 2 (ESD, ASD)

In [2]:
num_days = 5
num_courses = 12
num_rooms = 14
num_timeslots = 16
num_instructors = 6
num_students = 4

P1 = 2 # penalty for each day that instructor teach the same course for that day (for esd/ asd, that's teaching the two cohorts the same day)
P2 = 1 # penalty for students need to come on Monday

# construct instructor mat, each row is the courses by each instructor
I_r1 = [1 2]
I_r2 = [3 4]
I_r3 = [5 6]
I_r4 = [7 8]
I_r5 = [9 10]
I_r6 = [11 12]
instructor_mat = vcat(I_r1, I_r2, I_r3, I_r4, I_r5, I_r6)

# construct student mat, each row is the courses taken by each student group (cohort)
I_s1 = [1 3 5]
I_s2 = [2 4 6]
I_s3 = [7 9 11]
I_s4 = [8 10 12]
student_mat = vcat(I_s1,I_s2,I_s3,I_s4)

4×3 Matrix{Int64}:
 1   3   5
 2   4   6
 7   9  11
 8  10  12

In [11]:
m = Model(Gurobi.Optimizer)
@variable(m, x[1:num_courses,1:num_rooms,1:num_timeslots], Bin)
# @variable(m, d[1:num_instructors], Bin)
@variable(m, d[1:num_instructors,1:num_days], Bin)
@variable(m, s[1:num_students], Bin)

# constraint 1: Mon-Wed 1 class, Thu-Fri 1 class
for i in 1:num_courses
    @constraint(m, sum(x[i,j,k] for j in 1:num_rooms for k in 1:10)==1)
    @constraint(m, sum(x[i,j,k] for j in 1:num_rooms for k in 11:16)==1)
end

# constraint 2: in any room, any slot, only 1 course can take place
for j in 1:num_rooms
    for k in 1: num_timeslots
        @constraint(m,sum(x[i,j,k] for i in 1:num_courses)<=1)
    end
end

# constriant 3: students in the room not exceeding capacity
# pass

# constraint 4: in any slot, instructor teach 1 course
# for instructor in 1:num_instructors
#     course_of_this_instructor = instructor_mat[instructor,:]
#     for k in 1:num_timeslots
#         @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms)<=1)
#     end
# end

# instructor esd/ asd
for instructor in 1:num_instructors
    course_of_this_instructor = instructor_mat[instructor,:]
    for k in 1:num_timeslots
        @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms)<=1)
    end
end

# constraint 5: in any slot, student take 1 course
for student in 1:num_students
    course_of_this_student = student_mat[student,:]
    for k in 1:num_timeslots
        @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms)<=1)
    end
end

# constraint 6: courses everyday <= 2
for student in 1:num_students
    course_of_this_student = student_mat[student,:]
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 1:4)<=2)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 5:8)<=2)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 9:10)<=2)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 11:14)<=2)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 15:16)<=2)
end

# the additional constraint for objective
#P1
for instructor in 1:num_instructors
    course_of_this_instructor = instructor_mat[instructor,:]
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 1:4) - d[instructor,1] <= 1)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 5:8) - d[instructor,2] <= 1)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 9:10) - d[instructor,3] <= 1)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 11:14) - d[instructor,4] <= 1)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 15:16) - d[instructor,5] <= 1)
end
#P2
for student in 1:num_students
    course_of_this_student = student_mat[student,:]
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 1:4) - 2*s[student] <= 0)
end

@objective(m, Min, P1*sum(d[j,day] for j in 1:num_instructors for day in 1:num_days) + P2*sum(s[student] for student in 1:num_students))
print(m)
optimize!(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-19
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 462 rows, 2722 columns and 16834 nonzeros
Model fingerprint: 0xbcf560da
Variable types: 0 continuous, 2722 integer (2722 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 10.0000000
Presolve removed 184 rows and 1020 columns
Presolve time: 0.03s
Presolved: 278 rows, 1702 columns, 9766 nonzeros
Found heuristic solution: objective 6.0000000
Variable types: 0 continuous, 1702 integer (1702 binary)

Root relaxation: objective 0.000000e+00, 49 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf 

In [12]:
objective_value(m)

0.0

In [13]:
sum(value.(x[11,j,k] for j in 1:num_rooms for k in 1:num_timeslots))

2.0

In [30]:
for j in 1:num_rooms
    array1 = value.(x[12,j,:])
    indexArray = findall( x -> x == 1, array1 )
    println(j,indexArray)
end

1Int64[]
2Int64[]
3[5, 11]
4Int64[]
5Int64[]
6Int64[]
7Int64[]
8Int64[]
9Int64[]
10Int64[]
11Int64[]
12Int64[]
13Int64[]
14Int64[]


#### 2. Building 1 (ISTD, DAI)

In [31]:
num_days = 5
num_courses = 13
num_rooms = 13
num_timeslots = 16
num_instructors_3course = 1
num_instructors_2course = 4
num_instructors_1course = 2
num_instructors = num_instructors_1course+num_instructors_2course+num_instructors_3course
num_students_istd = 5
num_students_dai = 1
num_students = num_students_istd+num_students_dai

P1 = 2 # penalty for each day that instructor teach the same course for that day (for esd/ asd, that's teaching the two cohorts the same day)
P2 = 1

# construct instructor mat, each row is the courses by each instructor
I_r1 = [1 2 5]
I_r2 = [3 4]
I_r3 = [6 7]
I_r4 = [8 9]
I_r5 = [10 11]
I_r6 = [12]
I_r7 = [13]

instructor_mat_3course = I_r1
instructor_mat_2course = vcat(I_r2, I_r3, I_r4, I_r5)
instructor_mat_1course = vcat(I_r6, I_r7)

I_s1 = [1 6]
I_s2 = [2 7]
I_s3 = [3 8]
I_s4 = [4 9]
I_s5 = [5 10]
I_s6 = [11 12 13]
student_mat_istd = vcat(I_s1,I_s2,I_s3,I_s4, I_s5)
student_mat_dai = I_s6

1×3 Matrix{Int64}:
 11  12  13

In [32]:
m = Model(Gurobi.Optimizer)
@variable(m, x[1:num_courses,1:num_rooms,1:num_timeslots], Bin)
# instructor with 3 courses
@variable(m, d[1:num_instructors_3course, 1:num_days], Bin)
# instructor with 2 courses
@variable(m, f[1:num_instructors_2course,1:num_days], Bin)
@variable(m, s[1:num_students_istd], Bin)
@variable(m, g[1:num_students_dai], Bin)

# constraint 1: Mon-Tue 1 class, Wed-Fri 1 class
for i in 1:num_courses
    @constraint(m, sum(x[i,j,k] for j in 1:num_rooms for k in 1:8)==1)
    @constraint(m, sum(x[i,j,k] for j in 1:num_rooms for k in 9:16)==1)
end

# constraint 2: in any room, any slot, only 1 course can take place
for j in 1:num_rooms
    for k in 1: num_timeslots
        @constraint(m,sum(x[i,j,k] for i in 1:num_courses)<=1)
    end
end

# constriant 3: students in the room not exceeding capacity
# pass

# constraint 4: in any slot, instructor teach 1 course
# for instructor in 1:num_instructors
#     course_of_this_instructor = instructor_mat[instructor,:]
#     for k in 1:num_timeslots
#         @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms)<=1)
#     end
# end

# instructor 3 courses
for instructor in 1:num_instructors_3course
    course_of_this_instructor = instructor_mat_3course[instructor,:]
    for k in 1:num_timeslots
        @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms)<=1)
    end
end

# instructor 2 courses
for instructor in 1:num_instructors_2course
    course_of_this_instructor = instructor_mat_2course[instructor,:]
    for k in 1:num_timeslots
        @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms)<=1)
    end
end

# instructor 1 courses
for instructor in 1:num_instructors_1course
    course_of_this_instructor = instructor_mat_1course[instructor,:]
    for k in 1:num_timeslots
        @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms)<=1)
    end
end

# constraint 5: in any slot, student take 1 course
for student in 1:num_students_istd
    course_of_this_student = student_mat_istd[student,:]
    for k in 1:num_timeslots
        @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms)<=1)
    end
end

for student in 1:num_students_dai
    course_of_this_student = student_mat_dai[student,:]
    for k in 1:num_timeslots
        @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms)<=1)
    end
end

# constraint 6: courses everyday <= 2
for student in 1:num_students_istd
    course_of_this_student = student_mat_istd[student,:]
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 1:4)<=2)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 5:8)<=2)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 9:10)<=2)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 11:14)<=2)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 15:16)<=2)
end

for student in 1:num_students_dai
    course_of_this_student = student_mat_dai[student,:]
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 1:4)<=2)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 5:8)<=2)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 9:10)<=2)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 11:14)<=2)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 15:16)<=2)
end

# the additional constraint for objective
#P1
for instructor in 1:num_instructors_3course
    course_of_this_instructor = instructor_mat_3course[instructor,:]
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 1:4) - 2*d[instructor,1] <= 1)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 5:8) - 2*d[instructor,2] <= 1)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 9:10) - 2*d[instructor,3] <= 1)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 11:14) - 2*d[instructor,4] <= 1)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 15:16) - 2*d[instructor,5] <= 1)
end

for instructor in 1:num_instructors_2course
    course_of_this_instructor = instructor_mat_2course[instructor,:]
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 1:4) - 1*f[instructor,1] <= 1)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 5:8) - 1*f[instructor,2] <= 1)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 9:10) - 1*f[instructor,3] <= 1)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 11:14) - 1*f[instructor,4] <= 1)
    @constraint(m, sum(x[i,j,k] for i in course_of_this_instructor for j in 1:num_rooms for k in 15:16) - 1*f[instructor,5] <= 1)
end
# no need to consider the 1 course instructor. they will never take 2 courses on the same day

#P2
for student in 1:num_students_istd
    course_of_this_student = student_mat_istd[student,:]
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 15:16) - 2*s[student] <= 0)
end

for student in 1:num_students_dai
    course_of_this_student = student_mat_dai[student,:]
    @constraint(m, sum(x[i,j,k] for i in course_of_this_student for j in 1:num_rooms for k in 15:16) - 2*g[student] <= 0)
end

@objective(m, Min, P1*(sum(d[ins,day] for ins in 1:num_instructors_3course for day in 1:num_days) + sum(f[ins,day] for ins in 1:num_instructors_2course for day in 1:num_days)) + P2*(sum(s[student] for student in 1:num_students_istd)+sum(g[student] for student in 1:num_students_dai)))
print(m)
optimize!(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-19
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 503 rows, 2735 columns and 16177 nonzeros
Model fingerprint: 0xadeb0df5
Variable types: 0 continuous, 2735 integer (2735 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 13.0000000
Presolve removed 286 rows and 1373 columns
Presolve time: 0.06s
Presolved: 217 rows, 1362 columns, 6666 nonzeros
Found heuristic solution: objective 7.0000000
Variable types: 0 continuous, 1362 integer (1362 binary)

Root relaxation: objective 2.000000e+00, 52 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf 

In [33]:
objective_value(m)

3.0

In [36]:
sum(value.(x[13,j,k] for j in 1:num_rooms for k in 1:num_timeslots))

2.0

In [56]:
for j in 1:num_rooms
    array1 = value.(x[13,j,:])
    indexArray = findall( x -> x == 1, array1 )
    println(j,indexArray)
end

1[10]
2Int64[]
3Int64[]
4[7]
5Int64[]
6Int64[]
7Int64[]
8Int64[]
9Int64[]
10Int64[]
11Int64[]
12Int64[]
13Int64[]


In [39]:
value.(d)

1×5 Matrix{Float64}:
 0.0  1.0  -0.0  -0.0  -0.0

In [40]:
value.(f)

4×5 Matrix{Float64}:
 -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0

In [41]:
value.(s)

5-element Vector{Float64}:
 -0.0
  0.0
 -0.0
 -0.0
  1.0